In [11]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from model.Diffusion import Diffusion,gather
from model.Unet import Unet
from torch.utils.data import DataLoader,Dataset
from PIL import Image
from torchvision.transforms.functional import to_pil_image,resize
import numpy as np
import imageio


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_channel = 3
image_size = 160
unet = Unet()
model = Diffusion(unet,500,device=device)
n_steps = 500
print(device)

2
1
2
1
cuda:0


In [13]:
model_state_dict = torch.load('checkpoints/checkpoint.pth',weights_only=True)
print(model_state_dict['model_state_dict'])
model.load_state_dict(model_state_dict['model_state_dict'])

OrderedDict([('eps_model.image_proj.bias', tensor([-0.0693,  0.1642, -0.0540,  0.0299,  0.0492,  0.0673, -0.0293,  0.1633,
         0.0681,  0.1653,  0.1792, -0.0192,  0.0222,  0.0655,  0.0239,  0.0159,
        -0.1136, -0.1252,  0.0037, -0.0523, -0.0148,  0.0996, -0.0219, -0.0380,
        -0.1731, -0.0637,  0.1034,  0.0033, -0.0181, -0.0885, -0.0498,  0.1143,
         0.1574,  0.1564,  0.0955, -0.0981, -0.0777,  0.1154,  0.0662, -0.0719,
        -0.0071,  0.0304,  0.1180, -0.1198, -0.1385, -0.0418, -0.0412, -0.1285,
         0.0186, -0.1034, -0.0193,  0.0877, -0.1424,  0.1693,  0.0813, -0.1105,
        -0.1513, -0.0412, -0.1879, -0.1326,  0.1685,  0.0855,  0.0845, -0.0635],
       device='cuda:0')), ('eps_model.image_proj.weight_g', tensor([[[[0.6157]]],


        [[[0.6035]]],


        [[[0.4746]]],


        [[[0.6264]]],


        [[[0.5884]]],


        [[[0.5301]]],


        [[[0.6461]]],


        [[[0.5965]]],


        [[[0.6238]]],


        [[[0.5573]]],


        [[[0.505

<All keys matched successfully>

In [14]:
from torchvision.transforms import transforms
import imageio


def make_video(frames,path="video.mp4"):
    #20s 长度的视频
    writer = imageio.get_writer(path,fps=len(frames)//20)
    # 添加每个帧
    for f in frames:
        mean =(0.5,0.5,0.5)
        std =(0.5,0.5,0.5)
        denormalize = transforms.Normalize(mean=[-m/s for m, s in zip(mean, std)], std=[1/s for s in std])
        f = denormalize(f)
        f = f.clip(0,1)
        f = to_pil_image(resize(f,[368,368]))
        writer.append_data(np.array(f))
    writer.close()
def sample_animation(n_steps,n_frames=500):
    xt = torch.randn([1,image_channel,image_size,image_size],device=device)
    
    interval = n_steps // n_frames
    # 动画中的每一帧
    frames = []
    for t_inv in range(n_steps):
        t_ = n_steps - t_inv - 1
        t = xt.new_full((1,),t_,dtype=torch.long)
        if t_ % interval ==0:
            x0 = model.p_sample(xt,t,device)
            frames.append(x0)
        xt = model.p_sample(xt,t,device)
    make_video(frames)

In [15]:
model = model.to(device)
with torch.no_grad():
    sample_animation(n_steps)

ValueError: pic should be 2/3 dimensional. Got 4 dimensions.